# Make api call to retrieve education level of counties

In [2]:
from census import Census
from us import states

import numpy as np
import pandas as pd
import sys
import os
import json
import matplotlib.pyplot as plt
import requests

CENSUS_API_KEY = os.getenv('CENSUS_API_KEY')

from IPython.core.display import display

# pd.set_option('display.max_columns', None)
# pd.reset_optio('max_rows')
#np.set_printoptions(threshold=sys.maxsize)

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})


In [8]:
c = Census(CENSUS_API_KEY)

ESTIMATE_TOT = 'B06009_001E'
ESTIMATE_TOT_LESS_HS = 'B06009_002E'
ESTIMATE_TOT_HS = 'B06009_003E'
ESTIMATE_TOT_COLL_OR_ASS = 'B06009_004E'
ESTIMATE_TOT_BACH = 'B06009_005E'
ESTIMATE_TOT_GRAD = 'B06009_006E'

request = c.sf1.get('NAME', geo={'for': 'tract:*','in': 'state:{} county:170'.format(states.AK.fips)})

In [4]:
request = c.acs5.state_county(ESTIMATE_TOT, states.AL, '001')

In [5]:
request

[]

'70d51d7bfaa27460abe479195938c29899f06c18'